## Part 1: Preprocessing

In [1]:
# Import our dependencies
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import pandas as pd
import numpy as np
from tensorflow.keras.models import Model
import tensorflow.keras.metrics as metrics
from tensorflow.keras import layers

#  Import and read the attrition data
attrition_df = pd.read_csv('https://static.bc-edx.com/ai/ail-v-1-0/m19/lms/datasets/attrition.csv')
attrition_df.head()

2024-06-30 19:43:05.847268: I tensorflow/core/util/port.cc:113] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-06-30 19:43:06.053057: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX_VNNI FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


,Age,Attrition,BusinessTravel,Department,DistanceFromHome,Education,EducationField,EnvironmentSatisfaction,HourlyRate,JobInvolvement,...,PerformanceRating,RelationshipSatisfaction,StockOptionLevel,TotalWorkingYears,TrainingTimesLastYear,WorkLifeBalance,YearsAtCompany,YearsInCurrentRole,YearsSinceLastPromotion,YearsWithCurrManager
0,41,Yes,Travel_Rarely,Sales,1,2,Life Sciences,2,94,3,...,3,1,0,8,0,1,6,4,0,5
1,49,No,Travel_Frequently,Research & Development,8,1,Life Sciences,3,61,2,...,4,4,1,10,3,3,10,7,1,7
2,37,Yes,Travel_Rarely,Research & Development,2,2,Other,4,92,2,...,3,2,0,7,3,3,0,0,0,0
3,33,No,Travel_Frequently,Research & Development,3,4,Life Sciences,4,56,3,...,3,3,0,8,3,3,8,7,3,0
4,27,No,Travel_Rarely,Research & Development,2,1,Medical,1,40,3,...,3,4,1,6,3,3,2,2,2,2


In [2]:
# Determine the number of unique values in each column.
attrition_df.nunique()

Age                         43
Attrition                    2
BusinessTravel               3
Department                   3
DistanceFromHome            29
Education                    5
EducationField               6
EnvironmentSatisfaction      4
HourlyRate                  71
JobInvolvement               4
JobLevel                     5
JobRole                      9
JobSatisfaction              4
MaritalStatus                3
NumCompaniesWorked          10
OverTime                     2
PercentSalaryHike           15
PerformanceRating            2
RelationshipSatisfaction     4
StockOptionLevel             4
TotalWorkingYears           40
TrainingTimesLastYear        7
WorkLifeBalance              4
YearsAtCompany              37
YearsInCurrentRole          19
YearsSinceLastPromotion     16
YearsWithCurrManager        18
dtype: int64

In [3]:
attrition_df.columns

Index(['Age', 'Attrition', 'BusinessTravel', 'Department', 'DistanceFromHome',
       'Education', 'EducationField', 'EnvironmentSatisfaction', 'HourlyRate',
       'JobInvolvement', 'JobLevel', 'JobRole', 'JobSatisfaction',
       'MaritalStatus', 'NumCompaniesWorked', 'OverTime', 'PercentSalaryHike',
       'PerformanceRating', 'RelationshipSatisfaction', 'StockOptionLevel',
       'TotalWorkingYears', 'TrainingTimesLastYear', 'WorkLifeBalance',
       'YearsAtCompany', 'YearsInCurrentRole', 'YearsSinceLastPromotion',
       'YearsWithCurrManager'],
      dtype='object')

In [4]:
# Create y_df with the Attrition and Department columns
y_df = attrition_df[['Attrition', 'Department']]


In [5]:
# Create a list of at least 10 column names to use as X data
X_list = ['Age', 'DistanceFromHome', 'YearsAtCompany', 'YearsSinceLastPromotion',
        'Education', 'JobSatisfaction', 'OverTime', 'StockOptionLevel', 'WorkLifeBalance',
        'NumCompaniesWorked', 'MaritalStatus']


# Create X_df using your selected columns
X_df = attrition_df[X_list]

# Show the data types for X_df
X_df.dtypes


Age                         int64
DistanceFromHome            int64
YearsAtCompany              int64
YearsSinceLastPromotion     int64
Education                   int64
JobSatisfaction             int64
OverTime                   object
StockOptionLevel            int64
WorkLifeBalance             int64
NumCompaniesWorked          int64
MaritalStatus              object
dtype: object

In [6]:
# Split the data into training and testing sets
from sklearn.model_selection import train_test_split

Xtrain, Xtest, ytrain, ytest = train_test_split(X_df,y_df)


In [7]:
# Convert your X data to numeric data types however you see fit
# Add new code cells as necessary

from sklearn.preprocessing import LabelEncoder
# Convert overtime Yes/No to 1/0
le = LabelEncoder()
Xtrain.loc[:,'overtime_encoded'] = le.fit_transform(Xtrain['OverTime'])
Xtest.loc[:,'overtime_encoded'] = le.fit_transform(Xtest['OverTime'])

In [8]:
# Encode Married / Single
le2 = LabelEncoder()
Xtrain.loc[:,'MaritalStatus_encoded'] = le2.fit_transform(Xtrain['MaritalStatus'])
Xtest.loc[:,'MaritalStatus_encoded'] = le2.fit_transform(Xtest['MaritalStatus'])

In [9]:
Xtrain.drop(columns=['OverTime', 'MaritalStatus'], inplace=True)
Xtest.drop(columns=['OverTime', 'MaritalStatus'], inplace=True)

In [10]:
# Create a StandardScaler
scaler = StandardScaler()

# Fit the StandardScaler to the training data
scaler.fit(Xtrain)

# Scale the training and testing data
Xtrain_scaled = scaler.transform(Xtrain)
Xtest_scaled = scaler.transform(Xtest)

In [11]:
print(f"Department {y_df['Department'].value_counts()}")
print(f"\nAttrition {y_df['Attrition'].value_counts()}")


Department Department
Research & Development    961
Sales                     446
Human Resources            63
Name: count, dtype: int64

Attrition Attrition
No     1233
Yes     237
Name: count, dtype: int64


In [12]:
# Create a OneHotEncoder for the Department column
from sklearn.preprocessing import OneHotEncoder
encoder_dept = OneHotEncoder(handle_unknown='ignore', sparse_output=False)

# Fit the encoder to the training data
encoder_dept.fit(ytrain[['Department']])

# Create two new variables by applying the encoder
# to the training and testing data
# Transform the training data
ytrain_dept_encoded = encoder_dept.transform(ytrain[['Department']])
#ytrain_dept_encoded_df = pd.DataFrame(ytrain_encoded, columns=encoder.get_feature_names_out(['Department']))

# Transform the testing data
ytest_dept_encoded = encoder_dept.transform(ytest[['Department']])
#ytest_dept_encoded_df = pd.DataFrame(ytest_encoded, columns=encoder.get_feature_names_out(['Department']))

y_dept_encoded_columns = encoder_dept.get_feature_names_out(['Department'])

ytest_dept_encoded

array([[0., 1., 0.],
       [0., 0., 1.],
       [0., 1., 0.],
       ...,
       [0., 1., 0.],
       [0., 0., 1.],
       [0., 0., 1.]])

In [13]:
# Create a OneHotEncoder for the Attrition column
encoder_attr = OneHotEncoder(handle_unknown='ignore', sparse_output=False)

# Fit the encoder to the training data
encoder_attr.fit(ytrain[['Attrition']])

# Create two new variables by applying the encoder
# to the training and testing data
ytrain_attr_encoded = encoder_attr.transform(ytrain[['Attrition']])
#ytrain_attr_encoded_df = pd.DataFrame(ytrain_encoded, columns=encoder.get_feature_names_out(['Department']))

# Transform the testing data
ytest_attr_encoded = encoder_attr.transform(ytest[['Attrition']])
#ytest_attr_encoded_df = pd.DataFrame(ytest_encoded, columns=encoder.get_feature_names_out(['Department']))

y_attr_encoded_columns = encoder_attr.get_feature_names_out(['Attrition'])

ytest_attr_encoded[:5,:]

array([[1., 0.],
       [1., 0.],
       [1., 0.],
       [1., 0.],
       [1., 0.]])

## Create, Compile, and Train the Model

In [14]:
# Find the number of columns in the X training data
# Note:  11 input columns
input_nodes = len(X_df.columns)

# Create the input layer
input_layer = layers.Input(shape=(input_nodes,), name='common_layer1')

# Try: Relu, Gelu, Elu, selu, Sigmoid, Tanh, Swish, Softsign

# Create at least two shared layers (3x input nodes)
dense_common_1  = layers.Dense(33, activation='softsign')(input_layer)
dense_shared    = layers.Dense(33, activation='softsign')(dense_common_1)

2024-06-30 19:43:11.126000: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:984] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2024-06-30 19:43:11.196992: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:984] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2024-06-30 19:43:11.197048: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:984] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2024-06-30 19:43:11.204198: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:984] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2024-06-30 19:43:11.204250: I external/local_xla/xla/stream_executor

In [15]:
# Create a branch for Department
# with a hidden layer and an output layer

# Create the hidden layer
dense_dept = layers.Dense(33, activation='relu')(dense_shared)

# Create the output layer
dept_output = layers.Dense( len(y_dept_encoded_columns), 
                            activation='sigmoid',
                            name='dept_output')(dense_dept)


In [16]:
# Create a branch for Attrition
# with a hidden layer and an output layer

# Create the hidden layer
dense_attr = layers.Dense(33, activation='sigmoid')(dense_shared)

# Create the output layer
attr_output = layers.Dense( len(y_attr_encoded_columns), 
                            activation='sigmoid',
                            name='attr_output')(dense_attr)

In [17]:
# Create the model
model = Model(inputs=input_layer, outputs=[
    dept_output,
    attr_output
])

# Compile the model
model.compile(optimizer='adam',
              loss={'dept_output': 'categorical_crossentropy',
                    'attr_output': 'categorical_crossentropy'},
              metrics={'dept_output': [metrics.CategoricalAccuracy(), metrics.Precision(), metrics.Recall()],
                       'attr_output': [metrics.CategoricalAccuracy(), metrics.Precision(), metrics.Recall()]})

# Summarize the model
model.summary()

Model: "functional_1"

┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)        ┃ Output Shape      ┃    Param # ┃ Connected to      ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━┩
│ common_layer1       │ (None, 11)        │          0 │ -                 │
│ (InputLayer)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense (Dense)       │ (None, 33)        │        396 │ common_layer1[0]… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense_1 (Dense)     │ (None, 33)        │      1,122 │ dense[0][0]       │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense_2 (Dense)     │ (None, 33)        │      1,122 │ dense_1[0][0]     │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense_3 (Dense)     │ (None, 33)        │      1,122 │ dense_1[0][0]     │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dept_output (Dense) │ (None, 3)         │        102 │ dense_2[0][0]     │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ attr_output (Dense) │ (None, 2)         │         68 │ dense_3[0][0]     │
└─────────────────────┴───────────────────┴────────────┴───────────────────┘

 Total params: 3,932 (15.36 KB)

 Trainable params: 3,932 (15.36 KB)

 Non-trainable params: 0 (0.00 B)

In [18]:
# Train the model
model.fit(
    Xtrain,
    {
        'dept_output': ytrain_dept_encoded,
        'attr_output': ytrain_attr_encoded
    },
    epochs=10,  # You can adjust the number of epochs based on your needs
    batch_size=32,  # You can adjust the batch size based on your available memory
    validation_split=0.2  # You can specify the validation split if you have a separate validation set
)


Epoch 1/10


I0000 00:00:1719776593.883508   31046 service.cc:145] XLA service 0x7f95840245c0 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
I0000 00:00:1719776593.883634   31046 service.cc:153]   StreamExecutor device (0): NVIDIA GeForce RTX 3060 Laptop GPU, Compute Capability 8.6
2024-06-30 19:43:13.959626: I tensorflow/compiler/mlir/tensorflow/utils/dump_mlir_util.cc:268] disabling MLIR crash reproducer, set env var `MLIR_CRASH_REPRODUCER_DIRECTORY` to enable.
2024-06-30 19:43:14.222685: I external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:465] Loaded cuDNN version 8906


15/28 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - attr_output_categorical_accuracy: 0.8312 - attr_output_precision_1: 0.5022 - attr_output_recall_1: 0.9992 - dept_output_categorical_accuracy: 0.2716 - dept_output_precision: 0.2436 - dept_output_recall: 0.2086 - loss: 1.7223             

I0000 00:00:1719776596.073370   31046 device_compiler.h:188] Compiled cluster using XLA!  This line is logged at most once for the lifetime of the process.


28/28 ━━━━━━━━━━━━━━━━━━━━ 7s 111ms/step - attr_output_categorical_accuracy: 0.8350 - attr_output_precision_1: 0.5364 - attr_output_recall_1: 0.9780 - dept_output_categorical_accuracy: 0.3874 - dept_output_precision: 0.3714 - dept_output_recall: 0.3412 - loss: 1.6069 - val_attr_output_categorical_accuracy: 0.8507 - val_attr_output_precision_1: 0.8507 - val_attr_output_recall_1: 0.8507 - val_dept_output_categorical_accuracy: 0.6697 - val_dept_output_precision: 0.6667 - val_dept_output_recall: 0.6697 - val_loss: 1.2171
Epoch 2/10
28/28 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - attr_output_categorical_accuracy: 0.8151 - attr_output_precision_1: 0.8151 - attr_output_recall_1: 0.8151 - dept_output_categorical_accuracy: 0.6392 - dept_output_precision: 0.6336 - dept_output_recall: 0.6495 - loss: 1.2716 - val_attr_output_categorical_accuracy: 0.8507 - val_attr_output_precision_1: 0.8507 - val_attr_output_recall_1: 0.8507 - val_dept_output_categorical_accuracy: 0.6697 - val_dept_output_precision: 0.66

In [19]:
# Evaluate the model with the testing data
results = model.evaluate(np.array(Xtest), {
        'dept_output': ytest_dept_encoded,
        'attr_output': ytest_attr_encoded
    })

results

12/12 ━━━━━━━━━━━━━━━━━━━━ 1s 64ms/step - attr_output_categorical_accuracy: 0.8176 - attr_output_precision_1: 0.7756 - attr_output_recall_1: 0.8581 - dept_output_categorical_accuracy: 0.6357 - dept_output_precision: 0.5933 - dept_output_recall: 0.7009 - loss: 1.2308


[1.1964138746261597,
 0.83423912525177,
 0.7906404137611389,
 0.8722826242446899,
 0.6385869383811951,
 0.5976744294166565,
 0.698369562625885]

In [20]:
# Print the accuracy for both department and attrition
print(f"Loss: {results[0]}")

#print(f" output_activation: {output_activation}")
print(f"           Attrition   Department")
print(f"Accuracy:  {results[1]:.4f}      {results[4]:.4f}     The proportion of correctly predicted instances out of the total instance")
print(f"Precision: {results[2]:.4f}      {results[5]:.4f}     The ratio of true positive predictions to the total number of positive predictions")
print(f"Recall:    {results[3]:.4f}      {results[6]:.4f}     The ratio of true positive predictions to the total number of actual positives")


Loss: 1.1964138746261597
           Attrition   Department
Accuracy:  0.8342      0.6386     The proportion of correctly predicted instances out of the total instance
Precision: 0.7906      0.5977     The ratio of true positive predictions to the total number of positive predictions
Recall:    0.8723      0.6984     The ratio of true positive predictions to the total number of actual positives


In [21]:
# Loss: 5.376798152923584
#  output_activation: linear
#            Attrition   Department
# Accuracy:  0.8587      0.6413     The proportion of correctly predicted instances out of the total instance
# Precision: 0.8522      0.5858     The ratio of true positive predictions to the total number of positive predictions
# Recall:    0.8614      0.6957     The ratio of true positive predictions to the total number of actual positives

# Loss: 1.1802712678909302
# hidden_activation: softsign    output_activation: sigmoid
#            Attrition   Department
# Accuracy:  0.8587      0.6413     The proportion of correctly predicted instances out of the total instance
# Precision: 0.8725      0.5128     The ratio of true positive predictions to the total number of positive predictions
# Recall:    0.8370      0.8152     The ratio of true positive predictions to the total number of actual positives


# Loss: 1.1633143424987793
# hidden_activation: relu    output_activation: relu
#            Attrition   Department
# Accuracy:  0.8614      0.6413     The proportion of correctly predicted instances out of the total instance
# Precision: 0.8346      0.4783     The ratio of true positive predictions to the total number of positive predictions
# Recall:    0.8777      0.9565     The ratio of true positive predictions to the total number of actual positives


# Loss: 1.1592885255813599
# hidden_activation: sigmoid    output_activation: sigmoid
#            Attrition   Department
# Accuracy:  0.8478      0.6413     The proportion of correctly predicted instances out of the total instance
# Precision: 0.6294      0.5212     The ratio of true positive predictions to the total number of positive predictions
# Recall:    0.9783      0.8370     The ratio of true positive predictions to the total number of actual positives

In [22]:
print(f"Department {y_df['Department'].value_counts()}")
print(f"\nAttrition {y_df['Attrition'].value_counts()}")

Department Department
Research & Development    961
Sales                     446
Human Resources            63
Name: count, dtype: int64

Attrition Attrition
No     1233
Yes     237
Name: count, dtype: int64


# Summary

In the provided space below, briefly answer the following questions.

1. Is accuracy the best metric to use on this data? Why or why not?
2. What activation functions did you choose for your output layers, and why?
3. Can you name a few ways that this model might be improved?

YOUR ANSWERS HERE

1. Best metric to use:
    - Note:  Data is imbalanced for both Department and Attrition.
    - Accuracy is better for balanced data.
    - **Precision, Recall, and F1 Score** are better for imbalanced data.

2. Activation fucntion
    - I experimented with the following:
      - **Shared layers**: Relu, Gelu, Prelu, Elu, selu, Sigmoid, Tanh, Swish, Softsign.  Selu and Softsign performed the better than the others with **softsign being the best**.
      - **Output hidden layer**: **relu was best for attrition** and **softsign or sigmoid was best for department**.
      - **Output layer**: Sigmoid is worked well for both. But Linear also worked well and significantly increased precision for Department.
      - This combination also reduced the loss for 7.5+ to 1.2

3. Improve:
    - Try alternative Features such as: DistanceFromHome, StockOptionLevel, TotalWorkingYears, YearsInCurrentRole
    - Define a programatic method to try different activation functions.
    - Try a different optimizer: Nadam, RMSprop, SGD, Adagrad